In [2]:
import pandas as pd

wczytanie plików

In [3]:
stop_times = pd.read_csv('../data/stop_times.txt')
stops = pd.read_csv('../data/stops.txt')
trips = pd.read_csv('../data/trips.txt')
routes = pd.read_csv('../data/routes.txt')
routes.head()

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
0,route_31,agency_1,1,NaN,NaN,900,NaN,NaN,NaN
1,route_43,agency_1,19,NaN,NaN,900,NaN,NaN,NaN
2,route_46,agency_1,22,NaN,NaN,900,NaN,NaN,NaN
3,route_53,agency_1,62,NaN,NaN,900,NaN,NaN,NaN
4,route_54,agency_1,64,NaN,NaN,900,NaN,NaN,NaN


usunięcie niepotrzebnych rzeczy z plików

In [4]:
stop_times = stop_times.drop(columns=['stop_headsign','pickup_type', 'drop_off_type', 'shape_dist_traveled', 'timepoint'])
stop_times = stop_times[stop_times['trip_id'].str.contains('_service_1', na=False)]
stops = stops.drop(columns=['stop_code','stop_desc', 'zone_id', 'stop_url', 'location_type', 'parent_station', 'stop_timezone', 'wheelchair_boarding'])
trips = trips.drop(columns=['trip_short_name', 'shape_id', 'wheelchair_accessible', 'block_id'])
trips = trips[trips['trip_id'].str.contains('_service_1', na=False)]

routes = routes.drop(columns=['route_long_name', 'route_desc', 'route_type', 'route_url', 'route_color', 'route_text_color', 'agency_id'])

In [5]:
stop_times.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence
0,block_7_trip_1_service_1,04:59:00,04:59:00,stop_272_46429,2
1,block_7_trip_1_service_1,05:02:00,05:02:00,stop_256_43419,3
2,block_7_trip_1_service_1,05:03:00,05:03:00,stop_257_43519,4
3,block_7_trip_1_service_1,05:04:00,05:04:00,stop_342_268519,5
4,block_7_trip_1_service_1,05:05:00,05:05:00,stop_258_44219,6


In [6]:
stops.head()

,stop_id,stop_name,stop_lat,stop_lon
0,stop_183_6119,Bratysławska,50.083730,19.934160
1,stop_183_6129,Bratysławska,50.084645,19.933583
2,stop_184_6319,Krowodrza Górka,50.088727,19.931257
3,stop_186_7019,Dworzec Towarowy,50.075988,19.940292
4,stop_186_7029,Dworzec Towarowy,50.075943,19.940326


In [7]:
trips.head()

,trip_id,route_id,service_id,trip_headsign,direction_id
0,block_7_trip_1_service_1,route_942,service_1,Wzgórza K.,0
1,block_7_trip_2_service_1,route_942,service_1,Krowodrza Górka,1
2,block_7_trip_3_service_1,route_942,service_1,Wzgórza K.,0
3,block_7_trip_4_service_1,route_942,service_1,Krowodrza Górka,1
4,block_7_trip_5_service_1,route_942,service_1,Wzgórza K.,0


In [8]:
routes.head()

,route_id,route_short_name
0,route_31,1
1,route_43,19
2,route_46,22
3,route_53,62
4,route_54,64


Dodanie liczy porządkowej do przystanku

In [9]:
stops.columns = stops.columns.str.strip()
stops['stop_name'] = stops['stop_name'].str.replace(r'\s+', '', regex=True)
stops['stop_name'] = stops.groupby('stop_name').cumcount().add(1).astype(str).radd(stops['stop_name'])
stops.head()

,stop_id,stop_name,stop_lat,stop_lon
0,stop_183_6119,Bratysławska1,50.083730,19.934160
1,stop_183_6129,Bratysławska2,50.084645,19.933583
2,stop_184_6319,KrowodrzaGórka1,50.088727,19.931257
3,stop_186_7019,DworzecTowarowy1,50.075988,19.940292
4,stop_186_7029,DworzecTowarowy2,50.075943,19.940326


In [10]:
trips = trips.merge(routes, on='route_id')
trips = trips.drop(columns=['route_id'])
trips.head()

,trip_id,service_id,trip_headsign,direction_id,route_short_name
0,block_7_trip_1_service_1,service_1,Wzgórza K.,0,5
1,block_7_trip_2_service_1,service_1,Krowodrza Górka,1,5
2,block_7_trip_3_service_1,service_1,Wzgórza K.,0,5
3,block_7_trip_4_service_1,service_1,Krowodrza Górka,1,5
4,block_7_trip_5_service_1,service_1,Wzgórza K.,0,5


zamiana w stop_times id przystanku z nazwą

In [11]:
trips.columns = trips.columns.str.strip()

stop_times = stop_times.merge(stops[['stop_id', 'stop_name']], on='stop_id', how='left')

stop_times['stop_id'] = stop_times['stop_name']
stop_times = stop_times.drop(columns=['stop_name'])

stop_times = stop_times.merge(trips[['trip_id', 'direction_id', 'route_short_name']], on='trip_id', how='left')

stop_times.head(10)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,direction_id,route_short_name
0,block_7_trip_1_service_1,04:59:00,04:59:00,Elektromontaż(nż)2,2,0,5
1,block_7_trip_1_service_1,05:02:00,05:02:00,Wiadukty1,3,0,5
2,block_7_trip_1_service_1,05:03:00,05:03:00,Darwina1,4,0,5
3,block_7_trip_1_service_1,05:04:00,05:04:00,Jarzębiny1,5,0,5
4,block_7_trip_1_service_1,05:05:00,05:05:00,WzgórzaKrzesławickie1,6,0,5
5,block_7_trip_2_service_1,05:15:00,05:15:00,WzgórzaKrzesławickie1,1,1,5
6,block_7_trip_2_service_1,05:16:00,05:16:00,Jarzębiny2,2,1,5
7,block_7_trip_2_service_1,05:17:00,05:17:00,Darwina2,3,1,5
8,block_7_trip_2_service_1,05:18:00,05:18:00,Wiadukty2,4,1,5
9,block_7_trip_2_service_1,05:20:00,05:20:00,Wańkowicza1,5,1,5


Wygenerowanie linii połączeń - connections_df

In [12]:
connections = []

for _, group in stop_times.groupby("trip_id"):
    group = group.sort_values(by="stop_sequence") 
    stops2 = group["stop_id"].tolist()

    for i in range(len(stops2) - 1):
        connections.append((stops2[i], stops2[i + 1]))

connections_df = pd.DataFrame(connections, columns=["from_stop", "to_stop"])
stops = stops[(stops.stop_name != 'PT1') & (stops.stop_name != 'PH1') & (stops.stop_name != 'Łagiewniki3')] 
connections_df = connections_df[(connections_df.from_stop != 'PT1') & (connections_df.from_stop != 'PH1') & (connections_df.from_stop != 'Łagiewniki3')]
connections_df = connections_df[(connections_df.to_stop != 'PT1') & (connections_df.to_stop != 'PH1') & (connections_df.to_stop != 'Łagiewniki3')]
connections_df.head()

,from_stop,to_stop
0,NowyBieżanówP+R1,Ćwiklińskiej2
1,Ćwiklińskiej2,NowyProkocim2
2,NowyProkocim2,Teligi2
3,Teligi2,ProkocimSzpital2
4,ProkocimSzpital2,Prokocim2


In [13]:
connections_df.drop_duplicates(inplace=True)
connections_df.size

824

In [14]:
from geopy.distance import geodesic

def calculate_length(from_stop, to_stop):
    coord1 = (from_stop['stop_lat'], from_stop['stop_lon'])
    coord2 = (to_stop['stop_lat'], to_stop['stop_lon'])
    return geodesic(coord1, coord2).meters

Wygenerowanie pliku network2.xml

In [15]:
import xml.etree.ElementTree as ET
from xml.dom import minidom

def generate_file():
    network = ET.Element("network")
    nodes = ET.SubElement(network, "nodes")
    
    # Tworzenie węzłów
    for index, row in stops.iterrows():
        node = ET.SubElement(nodes, "node", id=row["stop_name"], x=str(row["stop_lon"]), y=str(row["stop_lat"]))
    
    links = ET.SubElement(network, "links")
    
    # Tworzenie linków i dodawanie długości
    for index, row in connections_df.iterrows():
        from_stop = stops[stops["stop_name"] == row['from_stop']].iloc[0]
        to_stop = stops[stops["stop_name"] == row['to_stop']].iloc[0]

        # Obliczanie odległości między przystankami
        length = calculate_length(from_stop, to_stop)

        # Tworzenie linku i dodawanie atrybutów
        link = ET.SubElement(links, "link", 
                             id=f"{row['from_stop']}-{row['to_stop']}", 
                             from_=row['from_stop'],
                             to=row['to_stop'], 
                            #  modes="pt", 
                             freespeed="20.0",
                             capacity="99999.0",
                             permlanes="1",
                             length=str(length))  # Dodajemy obliczoną długość
        
        link.attrib['from'] = link.attrib.pop('from_')  # Poprawiamy 'from' na 'from_'

    tree = ET.ElementTree(network)
    xml_str = ET.tostring(network, encoding="utf-8")
    parsed_str = minidom.parseString(xml_str)
    formatted_xml = parsed_str.toprettyxml(indent="  ")

    # Dodanie deklaracji XML na początku pliku
    final_xml = '<?xml version="1.0" encoding="utf-8"?>\n'
    final_xml += '<!DOCTYPE network SYSTEM "http://matsim.org/files/dtd/network_v1.dtd">\n'
    final_xml += formatted_xml[23:]  # Usuwamy nadmiarowe linie na początku

    # Zapisz do pliku
    with open("../config/network3.xml", "w", encoding="utf-8") as f:
        f.write(final_xml)

generate_file()
    

## wygenerowanie linii 

In [16]:
stop_times.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,direction_id,route_short_name
0,block_7_trip_1_service_1,04:59:00,04:59:00,Elektromontaż(nż)2,2,0,5
1,block_7_trip_1_service_1,05:02:00,05:02:00,Wiadukty1,3,0,5
2,block_7_trip_1_service_1,05:03:00,05:03:00,Darwina1,4,0,5
3,block_7_trip_1_service_1,05:04:00,05:04:00,Jarzębiny1,5,0,5
4,block_7_trip_1_service_1,05:05:00,05:05:00,WzgórzaKrzesławickie1,6,0,5


In [17]:
def is_full_trip(trip_df):
    stop_sequence_range = set(range(1, trip_df['stop_sequence'].max() + 1))
    return set(trip_df['stop_sequence']) == stop_sequence_range

stop_times = stop_times.groupby('trip_id').filter(is_full_trip)

stop_times['route_short_name'] = stop_times.apply(
    lambda row: str(row['route_short_name']) + 'r' if row['direction_id'] == 0 else str(row['route_short_name']),
    axis=1
)

stop_times = stop_times.drop(columns=['direction_id'])

stop_times.head(10)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,route_short_name
5,block_7_trip_2_service_1,05:15:00,05:15:00,WzgórzaKrzesławickie1,1,5
6,block_7_trip_2_service_1,05:16:00,05:16:00,Jarzębiny2,2,5
7,block_7_trip_2_service_1,05:17:00,05:17:00,Darwina2,3,5
8,block_7_trip_2_service_1,05:18:00,05:18:00,Wiadukty2,4,5
9,block_7_trip_2_service_1,05:20:00,05:20:00,Wańkowicza1,5,5
10,block_7_trip_2_service_1,05:21:00,05:21:00,Cienista1,6,5
11,block_7_trip_2_service_1,05:23:00,05:23:00,TeatrLudowy1,7,5
12,block_7_trip_2_service_1,05:26:00,05:26:00,RondoKocmyrzowskieim.Ks.Gorzelanego2,8,5
13,block_7_trip_2_service_1,05:27:00,05:27:00,Bieńczycka1,9,5
14,block_7_trip_2_service_1,05:30:00,05:30:00,RondoCzyżyńskie2,10,5


Tylko pierwsze przystanki z danej linii

In [18]:
first_stops_times = stop_times[stop_times['stop_sequence'] == 1].copy()
first_stops_times['block_id'] = first_stops_times['trip_id'].apply(lambda x: x.split('_')[0] + '_' + x.split('_')[1])
unique_blocks = {block: i+1 for i, block in enumerate(first_stops_times['block_id'].unique())}
first_stops_times['block_id'] = first_stops_times['block_id'].map(unique_blocks)

first_stops_times = first_stops_times[['block_id', 'stop_id', 'arrival_time', 'departure_time', 'route_short_name']].drop_duplicates()
first_stops_times = first_stops_times.sort_values(by=['block_id', 'stop_id', 'departure_time'])

first_stops_times = first_stops_times.sort_values(by=['route_short_name', 'arrival_time'])

first_stops_times.head(20)

,block_id,stop_id,arrival_time,departure_time,route_short_name
28664,61,Salwator1,04:44:00,04:44:00,1
53145,113,Salwator1,05:05:00,05:05:00,1
6460,15,Salwator1,05:22:00,05:22:00,1
42007,90,Salwator1,05:37:00,05:37:00,1
52243,111,Salwator1,05:45:00,05:45:00,1
37504,80,Salwator1,05:53:00,05:53:00,1
20842,45,Salwator1,06:00:00,06:00:00,1
37975,81,Salwator1,06:11:00,06:11:00,1
95829,211,Salwator1,06:21:00,06:21:00,1
48910,104,Salwator1,06:30:00,06:30:00,1


Czasy przejazdów dla każdych linii

In [19]:
from datetime import timedelta

every_time_stops = stop_times.sort_values(by=["route_short_name", "stop_sequence"])
every_time_stops = every_time_stops.drop_duplicates(subset=["stop_sequence", "route_short_name"], keep='first')

def update_times(group):
    times = []
    
    first_time = pd.to_datetime(group['arrival_time'].iloc[0])
    times.append(("00:00:00", group['stop_id'].iloc[0]))  # Pierwszy przystanek to zawsze 00:00:00
    
    # Iterujemy przez pozostałe przystanki
    for i in range(1, len(group)):
        next_time = pd.to_datetime(group['arrival_time'].iloc[i])
        time_diff = next_time - first_time
        minutes_diff = time_diff.total_seconds() / 60  # Konwersja na minuty
        
        # Obliczamy nowy czas
        new_time = pd.to_datetime('00:00:00') + pd.to_timedelta(minutes_diff, unit='m')
        times.append((new_time.strftime('%H:%M:%S'), group['stop_id'].iloc[i]))
    
    return times

times_stop_dict = {}

# Grupowanie po linii i zapisywanie wyników w słowniku
for route, group in every_time_stops.groupby('route_short_name'):
    times_stop_dict[route] = update_times(group)

print(times_stop_dict)


{'1': [('00:00:00', 'Salwator1'), ('00:01:00', 'Salwator2'), ('00:02:00', 'Komorowskiego2'), ('00:04:00', 'Jubilat2'), ('00:07:00', 'Filharmonia2'), ('00:08:00', 'PlacWszystkichŚwiętych2'), ('00:10:00', 'PocztaGłówna3'), ('00:12:00', 'Starowiślna3'), ('00:13:00', 'HalaTargowa1'), ('00:17:00', 'RondoGrzegórzeckie1'), ('00:18:00', 'TeatrVariété1'), ('00:19:00', 'FrancescoNullo1'), ('00:20:00', 'Fabryczna1'), ('00:21:00', 'OfiarDąbia1'), ('00:22:00', 'Dąbie1'), ('00:23:00', 'Plaza1'), ('00:24:00', 'TAURONArenaKrakówal.Pokoju(nż)1'), ('00:25:00', 'M1al.Pokoju1'), ('00:27:00', 'Rondo308.Dywizjonu1'), ('00:28:00', 'Centralna1'), ('00:31:00', 'RondoCzyżyńskie3'), ('00:32:00', 'Bieńczycka2'), ('00:35:00', 'RondoKocmyrzowskieim.Ks.Gorzelanego3'), ('00:36:00', 'TeatrLudowy2'), ('00:37:00', 'Cienista2'), ('00:39:00', 'Wańkowicza2'), ('00:41:00', 'Wiadukty1'), ('00:42:00', 'Darwina1'), ('00:43:00', 'Jarzębiny1'), ('00:44:00', 'WzgórzaKrzesławickie1')], '10': [('00:00:00', 'Łagiewniki2'), ('00:01:0

na potrzeby generate_transit_schedule

In [20]:
transit_schedule_data = stop_times.sort_values(by=["route_short_name", "stop_sequence"])
transit_schedule_data = every_time_stops.drop_duplicates(subset=["stop_sequence", "route_short_name"], keep='first')

stop_coords = stops.set_index('stop_name')[['stop_lat', 'stop_lon']]
transit_schedule_data = transit_schedule_data.merge(stop_coords, left_on='stop_id', right_index=True, how='left')

transit_schedule_data.drop(columns=['trip_id', 'arrival_time', 'departure_time'], inplace=True)

transit_schedule_data.to_csv('../data/transit_schedule_data.csv', index=False)

generowanie pliku transitschedule.xml

In [21]:
def generate_transit_schedule():
    root = ET.Element("transitSchedule")
    transit_stops = ET.SubElement(root, "transitStops")

    stop_facilities = set()  # Zbiór unikalnych stopFacility (id, x, y, linkRefId)

    # Grupowanie po liniach (route_short_name)
    for route, group in transit_schedule_data.groupby("route_short_name"):
        group = group.sort_values(by="stop_sequence")

        # Iteracja przez wiersze, aby utworzyć stopFacility dla każdej pary
        for i in range(len(group) - 1):
            current_stop = group.iloc[i]
            next_stop = group.iloc[i + 1]

            stop_id = current_stop["stop_id"]
            next_stop_id = next_stop["stop_id"]
            lat, lon = current_stop["stop_lat"], current_stop["stop_lon"]
            stop_facility_id = f"{stop_id}-{next_stop_id}"
            link_ref_id = f"{stop_id}-{next_stop_id}"

            if stop_facility_id not in stop_facilities:
                stop_facilities.add(stop_facility_id)
                ET.SubElement(
                    transit_stops,
                    "stopFacility",
                    id=stop_facility_id,
                    x=str(lon),
                    y=str(lat),
                    linkRefId=link_ref_id
                )

        last_stop = group.iloc[-1]
        last_stop_id = last_stop["stop_id"]
        last_lat, last_lon = last_stop["stop_lat"], last_stop["stop_lon"]
        previous_stop_id = group.iloc[-2]["stop_id"]
        
        stop_facility_id = f"{last_stop_id}"
        link_ref_id = f"{previous_stop_id}-{last_stop_id}"
        
        if stop_facility_id not in stop_facilities:
            stop_facilities.add(stop_facility_id)
            ET.SubElement(
                transit_stops,
                "stopFacility",
                id=stop_facility_id,
                x=str(last_lon),
                y=str(last_lat),
                linkRefId=link_ref_id
            )
            
    # for route, time_stop in times_stop_dict.items():
    #     transit_lines = ET.SubElement(tra, "transitLines")
    #     transit_line 
    

    tree = ET.ElementTree(root)

    xml_str = minidom.parseString(ET.tostring(root)).toprettyxml(indent="    ")

    # with open("../config/transitschedule2.xml", "w", encoding="utf-8") as f:
    #     f.write(xml_str)

generate_transit_schedule()

In [22]:
print(times_stop_dict)

{'1': [('00:00:00', 'Salwator1'), ('00:01:00', 'Salwator2'), ('00:02:00', 'Komorowskiego2'), ('00:04:00', 'Jubilat2'), ('00:07:00', 'Filharmonia2'), ('00:08:00', 'PlacWszystkichŚwiętych2'), ('00:10:00', 'PocztaGłówna3'), ('00:12:00', 'Starowiślna3'), ('00:13:00', 'HalaTargowa1'), ('00:17:00', 'RondoGrzegórzeckie1'), ('00:18:00', 'TeatrVariété1'), ('00:19:00', 'FrancescoNullo1'), ('00:20:00', 'Fabryczna1'), ('00:21:00', 'OfiarDąbia1'), ('00:22:00', 'Dąbie1'), ('00:23:00', 'Plaza1'), ('00:24:00', 'TAURONArenaKrakówal.Pokoju(nż)1'), ('00:25:00', 'M1al.Pokoju1'), ('00:27:00', 'Rondo308.Dywizjonu1'), ('00:28:00', 'Centralna1'), ('00:31:00', 'RondoCzyżyńskie3'), ('00:32:00', 'Bieńczycka2'), ('00:35:00', 'RondoKocmyrzowskieim.Ks.Gorzelanego3'), ('00:36:00', 'TeatrLudowy2'), ('00:37:00', 'Cienista2'), ('00:39:00', 'Wańkowicza2'), ('00:41:00', 'Wiadukty1'), ('00:42:00', 'Darwina1'), ('00:43:00', 'Jarzębiny1'), ('00:44:00', 'WzgórzaKrzesławickie1')], '10': [('00:00:00', 'Łagiewniki2'), ('00:01:0

In [23]:
def create_transit_route(parent, route, times_stops, first_stops_times):
    transit_route = ET.SubElement(parent, "transitRoute", id=route)
    
    # Transport mode
    ET.SubElement(transit_route, "transportMode").text = "pt"

    # Route profile
    route_profile = ET.SubElement(transit_route, "routeProfile")
    for i in range(len(times_stops) - 1):
        time, stop = times_stops[i]
        next_time, next_stop = times_stops[i + 1]
        ET.SubElement(route_profile, "stop", 
                      refId=f"{stop}-{next_stop}", 
                      departureOffset=time,
                      arrivalOffset=time,
                      awaitDeparture="true")
    
    # Ostatni przystanek
    last_time, last_stop = times_stops[-1]
    ET.SubElement(route_profile, "stop", 
                  refId=str(last_stop),
                  departureOffset=last_time,
                  arrivalOffset=last_time,
                  awaitDeparture="true")

    # Route links
    xml_route = ET.SubElement(transit_route, "route")
    for i in range(len(times_stops) - 1):
        _, stop = times_stops[i]
        _, next_stop = times_stops[i + 1]
        ET.SubElement(xml_route, "link", refId=f"{stop}-{next_stop}")

    # Departures
    xml_departures = ET.SubElement(transit_route, "departures")
    route_first_stops_times = first_stops_times[first_stops_times['route_short_name'] == route]
    for i, (_, row) in enumerate(route_first_stops_times.iterrows()):
        ET.SubElement(xml_departures, "departure",
                      id=f"{route}_{i}",
                      departureTime=row['departure_time'],
                      vehicleRefId=str(row['block_id']))

def generate_transit_line(transit_schedule):
    for route, times_stops in times_stop_dict.items():
        if 'r' not in route:
            r_route = route + 'r'
            r_times_stops = times_stop_dict[r_route]
            
            # Tworzenie transitLine
            transit_line = ET.SubElement(transit_schedule, "transitLine", id=route)
            
            # Dodanie route i r_route
            create_transit_route(transit_line, route, times_stops, first_stops_times)
            create_transit_route(transit_line, r_route, r_times_stops, first_stops_times)

    xml_str = minidom.parseString(ET.tostring(transit_schedule)).toprettyxml(indent="    ")
    print(xml_str)


# Generowanie XML
generate_transit_line(ET.Element("transitSchedule"))
            

<?xml version="1.0" ?>
<transitSchedule>
    <transitLine id="1">
        <transitRoute id="1">
            <transportMode>pt</transportMode>
            <routeProfile>
                <stop refId="Salwator1-Salwator2" departureOffset="00:00:00" arrivalOffset="00:00:00" awaitDeparture="true"/>
                <stop refId="Salwator2-Komorowskiego2" departureOffset="00:01:00" arrivalOffset="00:01:00" awaitDeparture="true"/>
                <stop refId="Komorowskiego2-Jubilat2" departureOffset="00:02:00" arrivalOffset="00:02:00" awaitDeparture="true"/>
                <stop refId="Jubilat2-Filharmonia2" departureOffset="00:04:00" arrivalOffset="00:04:00" awaitDeparture="true"/>
                <stop refId="Filharmonia2-PlacWszystkichŚwiętych2" departureOffset="00:07:00" arrivalOffset="00:07:00" awaitDeparture="true"/>
                <stop refId="PlacWszystkichŚwiętych2-PocztaGłówna3" departureOffset="00:08:00" arrivalOffset="00:08:00" awaitDeparture="true"/>
                <stop refId="

In [24]:
def generate_transit_vehicles(num_trams):
    # Tworzenie głównego elementu
    vehicle_definitions = ET.Element("vehicleDefinitions", {
        "xmlns": "http://www.matsim.org/files/dtd",
        "xmlns:xsi": "http://www.w3.org/2001/XMLSchema-instance",
        "xsi:schemaLocation": "http://www.matsim.org/files/dtd http://www.matsim.org/files/dtd/vehicleDefinitions_v1.0.xsd"
    })
    
    vehicle_type = ET.SubElement(vehicle_definitions, "vehicleType", id="1")
    ET.SubElement(vehicle_type, "description").text = "pt"
    capacity = ET.SubElement(vehicle_type, "capacity")
    ET.SubElement(capacity, "seats", persons="50")
    ET.SubElement(capacity, "standingRoom", persons="50")
    ET.SubElement(vehicle_type, "length", meter="12")
    
    # Generowanie pojazdów
    for i in range(1, num_trams + 1):
        ET.SubElement(vehicle_definitions, "vehicle", id=str(i), type="1")
    
    # Formatowanie XML
    xml_str = minidom.parseString(ET.tostring(vehicle_definitions)).toprettyxml(indent="    ")
    with open("../config/transitVehicles2.xml", "w", encoding="utf-8") as f:
        f.write(xml_str)

# Przykładowe wywołanie
num_trams = first_stops_times['block_id'].nunique()
generate_transit_vehicles(num_trams)


In [25]:
# connections_df.to_csv('../data/connections2.csv', index=False)    

generate population

In [29]:
import xml.etree.ElementTree as ET
import xml.dom.minidom
import random
import json

def get_nodes():
    tree = ET.parse("../config/transitSchedule2.xml")
    root = tree.getroot()
    stops_dict = {}
    for stop in root.find("transitStops"):
        stop_id = stop.attrib["id"]
        x = float(stop.attrib["x"])
        y = float(stop.attrib["y"])
        stops_dict[stop_id] = {"x": x, "y": y}
    
    new_schedule = {}

    for line, stops in times_stop_dict.items():
        new_schedule[line] = []
        for i in range(len(stops)):
            time, stop_name = stops[i]
            
            if i < len(stops) - 1:
                next_stop_name = stops[i + 1][1]
                stop_id = f"{stop_name}-{next_stop_name}"
            else:
                stop_id = stop_name
            
            if stop_id in stops_dict:
                new_schedule[line].append({
                    "x": stops_dict[stop_id]["x"],
                    "y": stops_dict[stop_id]["y"],
                    "id": stop_id
                })
            else:
                print(f"Brak współrzędnych dla {stop_id}")

    return new_schedule
    

def generate_population_xml(new_schedule, num_people, output_filename):
    root = ET.Element("plans")
    
    for person_id in range(1, num_people + 1):
        person = ET.SubElement(root, "person", employed="no", age=str(random.randint(10, 60)), id=str(person_id))
        plan = ET.SubElement(person, "plan", selected="yes")
        
        
        start_time = random.randint(4 * 3600, 22 * 3600 + 1800)  # Losowa godzina między 4:00 a 22:30
        max_end_time = 24 * 3600  # Maksymalna godzina to 24:00
        end_time = min(start_time + random.randint(3600, 8 * 3600), max_end_time)
        
        # wylosuj linię z new_schedule
        line = random.choice(list(new_schedule.keys()))
        stops = new_schedule[line]
        
        # wylosuj przystanek początkowy(nie może byc ostatnim) i końcowy
        start_stop = random.choice(stops[:-1])
        end_stop = random.choice(stops[stops.index(start_stop) + 1:])
        
        
        
        ET.SubElement(plan, "act", end_time=f"{start_time//3600:02}:{(start_time%3600)//60:02}:00", x=str(start_stop['x']), y=str(start_stop['y']), type="home")
        ET.SubElement(plan, "leg", mode="pt")
        ET.SubElement(plan, "act", end_time=f"{end_time//3600:02}:{(end_time%3600)//60:02}:00", x=str(end_stop['x']), y=str(end_stop['y']), type="home")
    
    rough_string = ET.tostring(root, encoding="utf-8")
    reparsed = xml.dom.minidom.parseString(rough_string)
    formatted_xml = "\n".join([line for line in reparsed.toprettyxml(indent="  ").split("\n")[1:] if line.strip()])
    
    with open(output_filename, "w", encoding="utf-8") as f:
        f.write('<?xml version="1.0" encoding="utf-8"?>\n')
        f.write('<!DOCTYPE plans SYSTEM "http://www.matsim.org/files/dtd/plans_v4.dtd">\n')
        f.write(formatted_xml)


new_schedule = get_nodes()

num_people = 100000
generate_population_xml(new_schedule, num_people, "../config/population.xml")

Brak współrzędnych dla NowyBieżanówP+R1-NowyBieżanówP+R1
